In [ ]:
#Run this on Storage server (10.5.99.50)

In [164]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.optimizers import Adadelta
from keras.backend import get_session
import pyodbc

In [166]:
#Necessary functions
from collections import Counter 

def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

In [167]:
#Get file path to execute branCTclassCNN
BrainCT=pd.read_csv('C:/Users/Administrator/Desktop/ChulHyoungPark/filepath.csv')

In [182]:
BrainCT

,Unnamed: 0,file_path
0,0,E:\PACS\PACS\bkpnew15\20090716\E4264874\I00103...
1,1,E:\PACS\PACS\bkpnew15\20090716\E4264874\I00103...
2,2,E:\PACS\PACS\bkpnew15\20090716\E4264874\I00103...
3,3,E:\PACS\PACS\bkpnew15\20090716\E4264874\I00103...
4,4,E:\PACS\PACS\bkpnew15\20090716\E4264874\I00103...
...,...,...
129758,129758,H:\PACS\bkpnew6\20070508\E2467960\I00049651513...
129759,129759,H:\PACS\bkpnew6\20070508\E2467960\I00049651514...
129760,129760,H:\PACS\bkpnew6\20070508\E2467960\I00049651515...
129761,129761,H:\PACS\bkpnew6\20070508\E2467960\I00049651516...


In [ ]:
#Add series id to datafreme
BrainCT['series_id']=BrainCT['file_path'].map(lambda x: pydicom.dcmread(x).SeriesInstanceUID)

In [ ]:
#Run this code in storage server to unzip image files
for i in BrainCT[['file_path']]:
    os.system('dcmdjpeg ' + i + ' ' + i)

In [173]:
#connect to mssql
server = '128.1.99.58'
database = 'ID_XYLITOL_WHITE'
username = 'hihipch'
password = 'gks1dls!'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [174]:
#Load model & weight from 14th server (brainCTclassCNN)
from keras.models import model_from_json
json_file = open("C:/Users/Administrator/Desktop/ChulHyoungPark/model1.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("C:/Users/Administrator/Desktop/ChulHyoungPark/model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [176]:
topogram=[]
vrt=[]
errorfile=[]
for i in np.unique(BrainCT['series_id']):
    x_train=[]
    imageid=[]
    for j in BrainCT.loc[BrainCT['series_id'] == i]['file_path']:
        if len(BrainCT.loc[BrainCT['series_id'] == i]['file_path'])==1:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'topogram' where series_id="+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
            topogram.append(i)
        else:
            metadata=pydicom.filereader.dcmread(j)
            imageid.append(metadata.SOPInstanceUID)
            pixeldata=metadata.pixel_array
            pixeldata=cv2.resize(pixeldata, (224, 224))
            if pixeldata.shape==(224, 224, 3):
                updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = '3D reconstruction' where series_id="+"'"+i+"'"
                cursor.execute(updatequery)
                cnxn.commit()
                vrt.append(i)
                break
            else:
                pixeldata[pixeldata < 0] = 0
                mean, std = pixeldata.mean(), pixeldata.std()
                regularized = (np.clip((pixeldata-mean)/std, -2.0, 2.0)+2.0)/4.0*255
                pixeldata = regularized.astype(np.uint8)
                backtorgb = cv2.cvtColor(pixeldata,cv2.COLOR_GRAY2RGB)
                mean, std = backtorgb.mean(), backtorgb.std()
                regularized = (np.clip((backtorgb-mean)/std, -2.0, 2.0)+2.0)/4.0
                x_train.append(regularized)
    try:
        x_train=np.array(x_train)
        yhat_train=loaded_model.predict(x_train, batch_size=1)
        prediction=[]
        for k in yhat_train:
            prediction.append(list(k).index(k.max()))
        print(most_frequent(prediction))

        if most_frequent(prediction)==0:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'MIP', anatomical_plane = 'axial' where series_id="+ "'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
        elif most_frequent(prediction)==1:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'post-contrast', anatomical_plane = 'axial' where series_id="+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
        elif most_frequent(prediction)==2:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'pre-contrast', anatomical_plane = 'axial' where series_id="+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
        elif most_frequent(prediction)==3:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'MIP', anatomical_plane = 'coronal' where series_id="+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
        elif most_frequent(prediction)==4:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'pre-contrast', anatomical_plane = 'coronal' where series_id="+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
        elif most_frequent(prediction)==5:
            updatequery="update [ID_XYLITOL_WHITE].[dbo].[RADIOLOGY_IMAGE] SET series_type = 'MIP', anatomical_plane = 'sagittal' where series_id=qwer"+"'"+i+"'"
            cursor.execute(updatequery)
            cnxn.commit()
    except:
        errorfile.append(i)

2
2
1
2
2
1
2
1
2
2
2
2
2
2
1
2
2
2
2
1
2
2
2
1
2
2
2
2
2
1
2
2
2
2
3
2
1
2
1
2
2
2
2
1
4
2
2
2
2
2
4
2
2
1
2
1
2
1
2
1
2
1
2
2
2
1
2
2
1
4
2
1
2
1
2
4
2
2
2
2
1
2
1
2
1
2
1
2
1
2
1
2
2
1
2
2
2
1
4
2
1
2
1
4
2
2
2
1
4
2
2
1
4
2
2
1
4
2
4
2
1
2
1
2
1
4
2
1
2
1
2
4
2
2
4
2
2
4
2
2
2
1
2
2
1
4
2
1
2
1
2
2
1
2
1
4
2
4
2
1
4
2
4
2
1
2
1
2
1
4
2
1
2
1
2
1
2
2
1
2
1
2
2
1
2
1
2
1
1
2
1
2
1
2
1
1
2
1
2
2
2
4
2
1
2
2
1
2
2
1
2
1
4
2
1
2
2
4
2
1
0
3
0
1
0
3
5
0
0
0
3
5
0
0
1
1
3
5
3
5
2
1
1
1
3
5
0
0
0
3
5
2
0
0
3
5
0
0
3
5
2
2
0
0
0
3
5
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
3
3
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
0
0
0
3
5
2
1
0
1
0
5
3
2
1
2
1
2
0
0
0
3
5
2
1
2
1
2
1
2
2
1
2
2
2
1
2
0
0
0
3
5
2
1
2
1
2
1
0
0
0
3
5
2
2
1
2
1
2
2
0
0
0
3
5
2
2
1
2
2
2
2
1
2
2
2
1
2
1
2
2
0
0
0
3
5
2
1
2
1
2
0
0
0
3
5
2
2
2
2
1
2
1
2
0
0
0
3
5
2
0
0
3
5
2
1
2
0
0
3
5
2
1
2
0
0
3
5
2
4


In [181]:
print(len(errorfile))
print(len(topogram))
print(len(vrt))
print(len(np.unique(admin['series_id'])))
print(len(admin['series_id']))

752
308
259
1990
65752


In [ ]:
errortopo=[]
zip_object = zip(errorfile, topogram)
for list1_i, list2_i in zip_object:
    errortopo.append(list1_i-list2_i)

In [ ]:
len(errortopo)